# 期望最大算法

本笔记参考[知乎文章](https://zhuanlan.zhihu.com/p/36331115), [B站视频](https://b23.tv/av31906558/p2)以及李航<<统计学习方法>>

## Introduction

概率模型有时既含有观测变量，又含有隐变量或潜在变量。如果概率模型的变量都是观测变量，那么给定数据，可以直接用极大似然估计法或贝叶斯估计法计算模型参数，但是当模型含有隐变量时，就不能简单地使用这些估计方法。EM算法就是含有隐变量的概率模型参数的极大似然估计法或极大后验概率估计法，我们仅仅讨论MLE。

首先，不予以证明地给出EM算法的基本步骤

> 输入:观察变量数据Y，隐藏变量Z，联合分布$P(Y, Z|\theta)$，条件分布$P(Z|Y, \theta)$
>
> 输出: 模型参数$\theta$
>
> (1)选择参数的初始值$\theta^{(0)}$, 开始迭代
>
> (2)E步: 记$\theta^{(i)}$为第i次迭代参数$\theta$的估计值，在第i + 1次迭代的E步，计算
> $$
\begin{aligned}
Q(\theta, \theta^{(i)}) & = E_Z\big[\log P(Y, Z| \theta) 
\big|Y, \theta^{(i)}\big]\\
& = \displaystyle\sum_Z\log P(Y, Z|\theta)P(Z|Y, \theta^{(i)})
\end{aligned}
$$
> 这里$P(Z|Y, \theta^{(i)})$是给定观测数据$Y$和当前的参数估计$\theta^{(i)}$下隐变量$Z$的条件概率分布。
>
> (3) M步: 求使$Q(\theta, \theta^{(i)})$极大化的$\theta$，确定第$i +1$次迭代的估计值$\theta^{(i + 1)}$
>
> (4) 重复第(2)步和第(3)步，直到收敛

## 基于KL散度推导

EM的最终目的是使得$P(Y|\theta)$能够取得最大值， 即$\log P(Y|\theta)$取得最大值
$$
\begin{aligned}
\log P(Y|\theta) &= \log\frac{P(Y,Z|\theta)}{P(Z|Y, \theta)}\\
& = \log P(Y,Z|\theta) - \log P(Z|Y, \theta)\\
&= \log\frac{P(Y,Z|\theta)}{q(Z)} - \log\frac{P(Z|Y, \theta)}{q(Z)} \\
\end{aligned}\tag{1}
$$

其中$q(Z)$为隐变量的某种分布，有$\displaystyle\int_Zq(Z)dZ = 1$,对公式(1)左侧求关于$q(Z)$的期望
$$
\displaystyle\int_Z \log P(Y|\theta)q(Z)dZ =\log P(Y|\theta) \displaystyle\int_Z q(Z)dZ = \log P(Y|\theta) \tag{2}\\
$$

那么公式(1)可以改写成
$$
\log P(Y|\theta) =\displaystyle\int_Zq(Z)\log\frac{P(Y,Z|\theta)}{q(Z)}dZ + \int_Zq(Z)\log\frac{q(Z)}{P(Z|Y, \theta)}dZ \tag{3}
$$

右侧的第二个积分式就是KL散度，其取值非负。所以得到
$$
\log P(Y|\theta) \ge\displaystyle\int_Zq(Z)\log\frac{P(Y,Z|\theta)}{q(Z)}dZ \tag{4}
$$

可以认为公式(4)的右侧是$\log P(Y|\theta)$的下界，那么我们需要做的事情就是最大化公式(4)右侧。

现在再回头，这个$q(Z)$是什么值呢?根据实际的应用，这个值取$P(Z|Y, \theta^{(i)})$是最合适的，那么EM的一次迭代优化过程就变成了
$$
\begin{aligned}
\theta^{(i + 1)} &= \arg\underset{\theta}{\max}
\displaystyle\int_ZP(Z|Y, \theta^{(i)})\log\frac{P(Y,Z|\theta)}{P(Z|Y, \theta^{(i)})}dZ \\
& = \arg\underset{\theta}{\max}
\displaystyle\int_ZP(Z|Y, \theta^{(i)})\big[\log P(Y,Z|\theta) - \log P(Z|Y, \theta^{(i)})\big]dZ\\
& = \arg\underset{\theta}{\max}
\displaystyle\int_ZP(Z|Y, \theta^{(i)})\log P(Y,Z|\theta)dZ\\
& = \arg\underset{\theta}{\max}Q(\theta, \theta^{(i)})
\end{aligned}\tag{5}
$$

## 基于Jensen Inequality

首先说明一下Jensen Inequality,对于一个凸函数$f(x)$
$$
E[f(x)] \ge f(E(x))\tag{6}
$$
当且仅当$x = E(x)$时公式(6)的等式成立。我们使用$q(Z)$表示样本中隐变量的某种分布。
$$
\begin{aligned}
\log P(Y|\theta) &= \log\displaystyle\sum_ZP(Y, Z|\theta)\\
&= \log \sum_Zq(Z)\frac{P(Y, Z|\theta)}{q(Z)}\\
& \ge \sum_Zq(Z)\log \frac{P(Y, Z|\theta)}{q(Z)}
\end{aligned}\tag{7}
$$
公式(7)中的不等式出现，就是利用了Jensen Inequality，对数函数$log(x)$是凹函数。

要使上式的不等式无限逼近，那么令
$$
\frac{P(Y, Z|\theta)}{q(Z)} = C\tag{8}
$$
根据公式(7)的前两个等号
$$
\displaystyle\sum_ZP(Y, Z|\theta) = \sum_Zq(Z)C\tag{9}
$$
公式(9)的右侧是一个常数的期望，结果仍然是这个常数，所以可以表示下式
$$
\displaystyle\sum_ZP(Y, Z|\theta) = C\tag{10}
$$


结合公式(8), (9), (10)得到
$$
\begin{aligned}
q(Z) &= \frac{P(Y, Z|\theta)}{\displaystyle\sum_ZP(Y, Z|\theta)}\\
&= \frac{P(Y, Z|\theta)}{P(Y|\theta)}\\
&= P(Z|Y, \theta)
\end{aligned}\tag{11}
$$

而$q(Z)$是固定模型参数后得到的值，所以实际上$q(Z) = P(Z|Y, \theta^{(i)})$, 那么一次迭代优化的目标为
$$
\begin{aligned}
\theta^{(i + 1)} 
&= \arg\underset{\theta}{\max}
\displaystyle\sum_ZP(Z|Y, \theta^{(i)})\log\frac{P(Y,Z|\theta)}{P(Z|Y, \theta^{(i)})} \\
& = \arg\underset{\theta}{\max}
\displaystyle\sum_ZP(Z|Y, \theta^{(i)})\big[\log P(Y,Z|\theta) - \log P(Z|Y, \theta^{(i)})\big]\\
& = \arg\underset{\theta}{\max}
\displaystyle\sum_ZP(Z|Y, \theta^{(i)})\log P(Y,Z|\theta)\\
& = \arg\underset{\theta}{\max}Q(\theta, \theta^{(i)})
\end{aligned}\tag{12}
$$

## EM的收敛性

要证明EM算法收敛，就要证明
$$
P(Y|\theta^{(i+1)}) \ge P(Y|\theta^{(i)})\tag{13}
$$
由于
$$
\log P(Y|\theta) = \log P(Y, Z|\theta)- \log P(Z|Y, \theta)\tag{14}
$$
令
$$
Q(\theta, \theta^{(i)}) = \displaystyle\sum_Z\log P(Y, Z|\theta)P(Z|Y, \theta^{(i)})\tag{15}
$$

$$
H(\theta, \theta^{(i)}) = \displaystyle\sum_Z\log P(Z| Y,\theta)P(Z|Y, \theta^{(i)})\tag{16}
$$


对公式(13)两侧同时对$P(Z|Y, \theta^{(i)})$积分， 对数似然函数可以写成

$$
\log P(Y|\theta) = Q(\theta, \theta^{(i)}) - H(\theta, \theta^{(i)})\tag{17}
$$

那么求解$\log P(Y|\theta^{(i+1)}) -\log P(Y|\theta^{(i)})$，公式(17)中的$\theta$ 分别取$\theta^{(i +1)}$和$\theta^{(i)}$

$$
\begin{aligned}
&\log P(Y|\theta^{(i+1)}) -\log P(Y|\theta^{(i)})\\
&= [Q(\theta^{(i+1)}, \theta^{(i)}) - Q(\theta^{(i)}, \theta^{(i)})] - 
[H(\theta^{(i+1)}, \theta^{(i)}) - H(\theta^{(i)}, \theta^{(i)})]
\end{aligned}\tag{18}
$$

由于$\theta^{(i+1)}$是使$Q(\theta, \theta^{(i)})$达到最大，所以有
$$
Q(\theta^{(i+1)}, \theta^{(i)}) - Q(\theta^{(i)}, \theta^{(i)}) \ge 0\tag{19}
$$
对于公式(18)中的第二项
$$
\begin{aligned}
H(\theta^{(i+1)}, \theta^{(i)}) - H(\theta^{(i)}, \theta^{(i)})
&= \displaystyle\sum_Z\bigg(\log \frac{P(Z|Y, \theta^{(i+1)})}{P(Z|Y, \theta^{(i)})}\bigg)P(Z|Y, \theta^{(i)})\\
&\le \log\bigg( \sum_Z\frac{P(Z|Y, \theta^{(i+1)})}{P(Z|Y, \theta^{(i)})}P(Z|Y, \theta^{(i)})\bigg)\\
&= log\bigg(\sum_ZP(Z|Y, \theta^{(i+1)})\bigg) = 0 
\end{aligned}\tag{20}
$$
此处的不等号由Jensen Inequality得到。

由公式(19)和(20)可知，公式(18)右端非负，即$L(\theta) = log P(Y|\theta^{(i)})$单调递增，加上$P(Y|\theta)$有界，所以EM算法是收敛的。